In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Read train and test datasets
pd.set_option('display.max_columns', 100)
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
train_df[train_df['Neighborhood'] == 'CollgCr'].mean()

Id                  726.840000
MSSubClass           43.300000
LotFrontage          71.682540
LotArea            9619.146667
Alley                      NaN
OverallQual           6.640000
OverallCond           5.240000
YearBuilt          1997.886667
YearRemodAdd       1999.140000
MasVnrArea           97.879195
BsmtFinSF1          456.586667
BsmtFinSF2           27.400000
BsmtUnfSF           673.600000
TotalBsmtSF        1157.586667
1stFlrSF           1164.626667
2ndFlrSF            315.873333
LowQualFinSF          0.000000
GrLivArea          1480.500000
BsmtFullBath          0.493333
BsmtHalfBath          0.020000
FullBath              1.753333
HalfBath              0.386667
BedroomAbvGr          2.820000
KitchenAbvGr          1.000000
TotRmsAbvGrd          6.353333
Fireplaces            0.393333
GarageYrBlt        1998.689189
GarageCars            2.006667
GarageArea          552.580000
WoodDeckSF          114.320000
OpenPorchSF          51.720000
EnclosedPorch         0.226667
3SsnPorc

In [4]:
train_df.Neighborhood.unique()

array(['CollgCr', 'Veenker', 'Crawfor', 'NoRidge', 'Mitchel', 'Somerst',
       'NWAmes', 'OldTown', 'BrkSide', 'Sawyer', 'NridgHt', 'NAmes',
       'SawyerW', 'IDOTRR', 'MeadowV', 'Edwards', 'Timber', 'Gilbert',
       'StoneBr', 'ClearCr', 'NPkVill', 'Blmngtn', 'BrDale', 'SWISU',
       'Blueste'], dtype=object)

In [12]:
neighborhoods = train_df.Neighborhood.unique()
average = [] * 25
for hood in neighborhoods:
    sale_prices = train_df[train_df['Neighborhood'] == hood]['SalePrice']
    ave_qual = train_df[train_df['Neighborhood'] == hood]['OverallQual']
    lot_area = train_df[train_df['Neighborhood'] == hood]['LotArea']
    bed_num = train_df[train_df['Neighborhood'] == hood]['BedroomAbvGr']
    totalSP = 0
    countSP = 0
    totalQ = 0
    countQ = 0
    totalLA = 0
    countLA = 0
    totalBR = 0
    countBR = 0
    
    for row in sale_prices:
        totalSP += row
        countSP += 1
        
    for row in ave_qual:
        totalQ += row
        countQ += 1
        
    for row in lot_area:
        totalLA += row
        countLA += 1
        
    for row in bed_num:
        totalBR += row
        countBR += 1
        
    print(hood, '  \tCount:', countSP , '  \t$',  round(totalSP / countSP, 2), '  \tAvg Quality:', round(totalQ / countQ, 2), '  \tAvg Lot Area:', round(totalLA / countLA, 2), '   \tBedr Num:', round(totalBR / countBR, 2))

CollgCr   	Count: 150   	$ 197965.77   	Avg Quality: 6.64   	Avg Lot Area: 9619.15    	Bedr Num: 2.82
Veenker   	Count: 11   	$ 238772.73   	Avg Quality: 6.73   	Avg Lot Area: 15606.0    	Bedr Num: 2.18
Crawfor   	Count: 51   	$ 210624.73   	Avg Quality: 6.27   	Avg Lot Area: 11809.69    	Bedr Num: 2.98
NoRidge   	Count: 41   	$ 335295.32   	Avg Quality: 7.93   	Avg Lot Area: 14218.9    	Bedr Num: 3.51
Mitchel   	Count: 49   	$ 156270.12   	Avg Quality: 5.59   	Avg Lot Area: 11624.29    	Bedr Num: 2.76
Somerst   	Count: 86   	$ 225379.84   	Avg Quality: 7.34   	Avg Lot Area: 7840.87    	Bedr Num: 2.67
NWAmes   	Count: 73   	$ 189050.07   	Avg Quality: 6.33   	Avg Lot Area: 11833.63    	Bedr Num: 3.29
OldTown   	Count: 113   	$ 128225.3   	Avg Quality: 5.39   	Avg Lot Area: 8207.06    	Bedr Num: 2.81
BrkSide   	Count: 58   	$ 124834.05   	Avg Quality: 5.05   	Avg Lot Area: 7360.41    	Bedr Num: 2.6
Sawyer   	Count: 74   	$ 136793.14   	Avg Quality: 5.03   	Avg Lot Area: 10459.28    	Bed

In [6]:
print(train_df.shape)
print(test_df.shape)

(1460, 81)
(1459, 80)
